In [ ]:
# Ipython 
import matplotlib.pyplot as plt
from sympy.physics.vector import init_vprinting
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
get_ipython().run_line_magic('matplotlib', 'inline')
get_ipython().run_line_magic('load_ext', 'autoreload')
get_ipython().run_line_magic('autoreload', '2')
init_vprinting(use_latex='mathjax', pretty_print=False)
# Sympy
import numpy as np
import sympy as sp
from sympy.physics.mechanics import dynamicsymbols
# Yams sympy
from welib.yams.yams_sympy       import YAMSRigidBody, YAMSInertialBody, YAMSFlexibleBody
from welib.yams.yams_sympy_model import YAMSModel

# Two masses in a row, bounded by two walls

NOTE: not using relative position between two masses
```

          |--> x1       |--> x2
/|                                    |\
/| - k1 - [ m1 ] - k2 - [ m2 ] - k3 - |\
/| - c1 - [    ] - c2 - [    ] - c3 - |\
/|                                    |\
```

In [ ]:
m1, m2, k1, k2, k3, c1, c2, c3 = sp.symbols('m1, m2, k1, k2, k3, c1, c2, c3')
gravity = sp.symbols('g')
time = dynamicsymbols._t
x1, x2 = dynamicsymbols('x_1, x_2')
x1d, x2d = dynamicsymbols('x1d, x2d')
v1, v2 = dynamicsymbols('x_1, x_2', 1)
# --- Bodies
ref = YAMSInertialBody('E') 
body1 = YAMSRigidBody('B_1', mass=m1) # Create a body named "B_1"
body2 = YAMSRigidBody('B_2', mass=m2) # Create a body named "B_2"
bodies=[body1, body2] 
# --- Connections
ref.connectTo(body1, type='Free', rel_pos=(x1, 0, 0))
ref.connectTo(body2, type='Free', rel_pos=(x2, 0, 0)) # NOTE: absolute
# --- DOFs and kinetic equations
coordinates = [x1, x2]
speeds      = [x1d, x2d]
kdeqsSubs = []
for q,qd in zip(coordinates, speeds):
    kdeqsSubs += [(qd, sp.diff(q, time))]
# --- Loads
body_loads = []
body_loads += [(body1, (body1.origin,  - k1 * x1 * ref.frame.x  - c1 * v1 * ref.frame.x ))]
body_loads += [(body2, (body2.origin,  - k3 * x2 * ref.frame.x  - c3 * v2 * ref.frame.x ))]
body_loads += [(body1, (body1.origin,    k2 * (x2-x1) * ref.frame.x  + c2 * (v2 - v1) * ref.frame.x ))]
body_loads += [(body2, (body2.origin,  - k2 * (x2-x1) * ref.frame.x  - c2 * (v2 - v1) * ref.frame.x ))]
# --- Model and equations of motions
model = YAMSModel(name='MSD', ref=ref, bodies=bodies, body_loads=body_loads, 
                  coordinates=coordinates, speeds=speeds, kdeqsSubs=kdeqsSubs, g_vect=-gravity*ref.frame.z)
EOM=model.to_EOM()
print('Equations of motions')
EOM.EOM
EOM.mass_forcing_form()
print('Mass matrix:')
EOM.M
print('Forcing term:')
EOM.F
M0, C0, K0, B0 = EOM = EOM.linearize(op_point=None, noAcc=True, noVel=False, extraSubs=None, simplify=True)
M0, C0, K0, B0

In [ ]:
# Export to python code
#print(EOM.toPython())
#EOM.savePython('_TuneMassDamper.py')